# Imports

In [1]:
suppressPackageStartupMessages({
    library('utils')
    library('vroom')
    library('tidyverse')
})


# Load and Process Data

In [2]:
df.temp = vroom("Data/GlobalLandTemperaturesByCity.csv", progress = F, show_col_types = FALSE)

In [4]:
df.temp %>% 
  mutate(
    date = as.Date(dt, "%Y-%m-%d"),
    year =  format(date, '%Y'),
    month = as.factor(format(date, "%b")),
    day = format(date, "%d")
    ) %>% 
    filter(!is.na(AverageTemperature)) %>% # Remove missing values
    group_by(year, City) %>% 
    mutate(nrow = n()) %>% 
    filter(nrow>11) %>%  # Remove any Cities that don't have complete 12 month data
    ungroup() %>% 
    filter(month == "Dec") %>% # Select a specific month
    group_by(year, City, Country, Latitude, Longitude) %>% 
    summarise(AvgTemp_year = mean(AverageTemperature), .groups="drop_last") %>% # Average the Average temperature into yearly avg temp
    ungroup() %>% 
    mutate( # Change South West indicators to negative positive
    sign_long = case_when(str_detect(Longitude, "W")~-1, T~1),
    sign_lat = case_when(str_detect(Latitude, "S")~-1, T~1),
    lat = sign_lat*as.numeric(str_remove(Latitude, "[A-Z]")), # Multiply the sign by the lat/long degree value
    long = sign_long*as.numeric(str_remove(Longitude, "[A-Z]"))
    ) %>% 
    select(c(year, City, Country, AvgTemp_year, lat, long)) %>%
    head()

year,City,Country,AvgTemp_year,lat,long
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1744,Birmingham,United Kingdom,3.712,52.24,-2.63
1744,Birmingham,United States,6.895,32.95,-87.13
1744,Brest,Belarus,-3.280,52.24,23.65
1744,Brest,France,9.092,49.03,-4.90
1744,Cambridge,United Kingdom,3.752,52.24,0.00
1744,Cambridge,United States,-5.335,42.59,-72.00
